In [1]:
import altair as alt
import numpy as np
import pandas as pd

import json

In [86]:
blue_mrtk = "#122973"
kpis_columns = ['shiftIdentifier', 'locationId', 'validFrom', 'validTo', 'oee',
       'oeeSetpoint', 'availability', 'efficiency', 'quality', 'fpy',
       'fpySetPoint', 'partCount', 'partCountSetPoint', 'countNIO',
       'productivity', 'station']

cycle = pd.read_json("./simulation/CycleTimes.json")
kpi =   pd.read_json("./simulation/KPI.json")

with open(f'DadosSimulados/TargetsExtraSim.json') as f:
    targets = json.load(f)
    
def getMetric(t,iskpi=False):
    scaleCor = alt.Scale(domain=[targets[t]*.9,targets[t]*.95,targets[t]],range=["green","yellow","red"] if not iskpi else ["red","yellow","green"])
    if (t in cycle.columns):
        v = cycle[t].iloc[len(cycle) - 1]
    else:
        v = kpi[t].iloc[len(kpi) - 1]
        
    return (alt.Chart().mark_text(fill="white").encode(
        text=alt.value([f"{t}: {v}",f"Target: {targets[t]}"]),
        size=alt.value(16)
    )) | (alt.Chart().mark_circle(fill="red").encode(
        size=alt.Size(f"mean({t}):Q", scale=alt.Scale(domain=[targets[t]*.9, targets[t]] * 2, range=[8000,100] if iskpi else [100, 8000])),
        fill=alt.Fill(f"mean({t}):Q", scale=scaleCor),
        tooltip=[f"mean({t}):Q"]
    ) + alt.Chart().mark_text(fill="white").encode(
        text=alt.Text(f"mean({t}):Q", format='.4'),
        opacity=alt.value(0)
    ))

def getOverview(t, title=""):
    return alt.Chart().mark_text(fill="white").encode(
        text=alt.value(f"{title}"),
        size=alt.value(16)
    ) & (alt.Chart().mark_circle(fill="red").encode(
        size=alt.Size(f"{t}:Q", scale=alt.Scale(range=[80,800])),
        text=alt.value(f"{title}"),
    ) + alt.Chart().mark_text(fill="white").encode(
        text=alt.Text(f"mean({t}):Q", format='.4'),
        opacity=alt.value(0)
    ))
    

def getConfigs(chart):
    return chart.configure(
        background=blue_mrtk
    ).transform_lookup(lookup="station",
        from_=alt.LookupData(data=kpi.iloc[[0]], key='station', fields=list(kpis_columns))
    ).configure_axis(
        titleColor="white",
        labelColor="white",
        labelAngle=-45
    ).configure_header(
        titleColor=blue_mrtk,
        titleFontSize=14,
        labelColor=blue_mrtk,
        labelFontSize=14
    ).configure_legend(
        strokeColor='white',
        labelColor='white',
        titleColor='white',
        padding=1,
        tickCount=2,
        disable=True
    ).configure_view(
        strokeWidth=0,
        width=100,
        height=100
    ).transform_filter(
        "datum.station == 260"
    )

getConfigs(alt.hconcat(
    (getMetric("totalCycleTime") & getMetric("processTime")), (getMetric("exitTime") & getMetric("changeTime")),  
    data=cycle.iloc[[0]]
))


alt.HConcatChart(...)

In [83]:
getConfigs(alt.hconcat(
    (getMetric("oee",True) & getMetric("fpy",True)), (getMetric("partCount",True) & getMetric("partCountSetPoint",True) & getMetric("productivity",True)),  
    data=cycle.iloc[[0]]
))

alt.HConcatChart(...)

In [4]:
getConfigs(alt.hconcat(
    getOverview("totalCycleTime", "Cycle Times"), getOverview("partCount", "Bottleneck"), getOverview("changeTime", "KPIs"),
    data=cycle.iloc[[0]]
))

alt.HConcatChart(...)

In [7]:
blue_mrtk = "#122973"

line = alt.Chart(cycle.reset_index()).mark_line(color="gold").encode(
    y='totalCycleTime:Q',
    x='index:Q'
).transform_filter(
    "datum.station == '260'"
)

rule_linechart = alt.Chart(cycle.reset_index()).mark_rule(color='red').encode(
    y='mean(totalCycleTime):Q',
    size=alt.value(6)
).transform_filter(
    "datum.station == '260'"
)

(line + rule_linechart).properties(width=600).configure(
    background=blue_mrtk
).configure_axis(
    titleColor="white",
    labelColor="white"
).configure_header(
    titleColor=blue_mrtk,
    titleFontSize=14,
    labelColor=blue_mrtk,
    labelFontSize=14
).transform_filter(
    "datum.index >= 0 && datum.index < 100"
)


alt.LayerChart(...)

In [90]:
blue_mrtk = "#122973"

line = alt.Chart(".simulation/KPI.json").mark_line(color="gold").encode(
    y='oee:Q',
    x='index:Q'
).transform_filter(
    "datum.station == '260'"
)
# https://raw.githubusercontent.com/tiagodavi70/dashboard_linha_montagem/main
rule_linechart = alt.Chart(".simulation/KPI.json").mark_rule(color='red').encode(
    y='mean(oee):Q',
    size=alt.value(6)
).transform_filter(
    "datum.station == '260'"
)

(line + rule_linechart).properties(width=600).configure(
    background=blue_mrtk
).configure_axis(
    titleColor="white",
    labelColor="white"
).configure_header(
    titleColor=blue_mrtk,
    titleFontSize=14,
    labelColor=blue_mrtk,
    labelFontSize=14
)
# .transform_filter(
#     "datum.index >= 0 && datum.index < 100"
# )


alt.LayerChart(...)